In [2]:
import os
import pandas as pd
import numpy as np
import math

In [2]:
data = pd.read_csv("data/diff/2019-02.csv")
data.head()

,time,0,1,2,3,4,5,6,7,8,...,255,256,257,258,259,260,261,262,263,264
0,2019-02-01 00:00:00,1,0,0,37,0,0,30,0,0,...,32,2,1,-1,13,-4,47,57,-28,13
1,2019-02-01 01:00:00,1,0,0,29,0,0,52,0,1,...,30,10,4,1,15,4,41,37,13,26
2,2019-02-01 02:00:00,1,0,0,14,0,1,33,0,0,...,10,4,0,1,15,7,21,9,4,11
3,2019-02-01 03:00:00,1,0,0,12,0,0,20,0,0,...,3,1,0,1,8,0,15,8,2,8
4,2019-02-01 04:00:00,7,0,1,8,0,0,10,0,1,...,3,0,3,1,1,3,-5,-9,1,10


In [3]:
days = int(data.shape[0] / 24)

In [4]:
data['day'] = pd.to_datetime(data['time']).dt.day
data['hour'] = pd.to_datetime(data['time']).dt.hour
data['date'] = pd.to_datetime(data['time']).dt.date

In [7]:
# 绘制一个月内部分区域的出租车热度图
from pyecharts import options as opts
from pyecharts.charts import HeatMap
from pyecharts.charts import Timeline

tl = (
    Timeline()
    .add_schema(play_interval=200,is_loop_play=False)
    )
    
x = [i for i in range(220,265)]
y = [i for i in range(24)]
for i in range(days):
    value = [[k, j, int(data.iloc[i*24+j][k+221])] for k in range(45) for j in range(24)]
    c = (
        HeatMap()
        .add_xaxis(x)
        .add_yaxis(str(data.iloc[i*24]['date']), y, value)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="2019年2月部分地区热力图"),
            visualmap_opts=opts.VisualMapOpts(min_=-30, max_=30),
        )
    )
    tl.add(c, "{}日".format(i+1))
tl.render("data/graph/2019-02.html")

'd:\\下载\\TLC\\2019-02.gif'

In [13]:
dist2 = pd.read_csv("data/dist/2019-02.csv")
dist1 = pd.read_csv("data/dist/2019-01.csv")
dist = dist1-dist2

In [15]:
# 绘制地区之间距离热度图
from pyecharts import options as opts
from pyecharts.charts import HeatMap

x = [i for i in range(265)]
y = [i for i in range(265)]
value = [[i, j, int(dist.iloc[i][j])] for i in range(265) for j in range(265)]
c = (
    HeatMap(init_opts=opts.InitOpts(width='1000px',height='900px'))
    .add_xaxis(x)
    .add_yaxis("distance", y, value)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各地区距离"),
        visualmap_opts=opts.VisualMapOpts(min_=-30, max_=30),
    )
    .render("data/graph/距离差.html")
)

In [56]:
data = pd.read_csv("data/diff/2019-01.csv")
data.head()

,time,0,1,2,3,4,5,6,7,8,...,255,256,257,258,259,260,261,262,263,264
0,2019-01-01 00:00:00,0,0,0,25,0,0,20,0,0,...,2,3,0,1,5,-6,44,-54,-68,19
1,2019-01-01 01:00:00,3,0,2,32,0,0,92,-1,2,...,-4,16,3,4,27,12,170,24,-6,45
2,2019-01-01 02:00:00,2,0,2,36,0,1,115,2,2,...,-29,12,6,0,37,15,168,67,-9,48
3,2019-01-01 03:00:00,8,0,4,35,0,0,102,0,4,...,20,9,7,0,36,34,87,45,12,53
4,2019-01-01 04:00:00,13,0,0,35,0,0,71,0,2,...,21,5,4,8,12,21,59,-1,1,22


In [57]:
import numpy as np
data=data.iloc[:,221:]
days = data.shape[0]//24
mean = data.iloc[0:24].values
for i in range(1,days):
    tmp = data.iloc[i*24:(i+1)*24].values
    mean = mean + tmp
mean = mean / days
mean = pd.DataFrame(mean)
mean.columns = range(220,265)
mean.to_csv("data/model/2019-01.csv")

In [3]:
def node_color(n):
    if n < 0:
        return "blue"
    if n > 0:
        return "red"
    return "yellow"

node_color(1)

'red'

In [13]:
from pyecharts import options as opts
from pyecharts.charts import Timeline
from pyecharts.charts import Graph

tl = Timeline()

node = pd.read_csv("data/model/diff.csv")
edge = np.load("data/model/graph.npy")

for k in range(24):
    nodes = [
        {
            "name": str(i+220),
            "value": node.iloc[k,i],
            "symbolSize": math.log(1+abs(node.iloc[k,i]),1.1),
            "itemStyle": {"normal": {"color": node_color(node.iloc[k,i])}},
        }
        for i in range(45)
    ]
    edges = []
    for i in range(45):
        for j in range(45):
            if edge[k,i,j] != 0:
                edges.append({"source":str(i+220), "target":str(j+220), "value":edge[k,i,j]})
    c = (
        Graph(init_opts=opts.InitOpts(width="1600px", height="800px"))
        .add(
            series_name="",
            nodes=nodes,
            links=edges,
            layout="circular",
            is_roam=True,
            is_focusnode=True,
            label_opts=opts.LabelOpts(is_show=False),
            linestyle_opts=opts.LineStyleOpts(width=0.5, curve=0.3, opacity=0.7),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="NPM Dependencies"))
    )
    tl.add(c, "{}时".format(k))
tl.render("data/graph/关系图.html")

'd:\\下载\\TLC\\data\\graph\\关系图.html'